In [1]:
import torch
import threading
import time
import pyrender
import numpy as np
import optimal_transport_modules
import matplotlib.pyplot as plt
import trimesh
from optimal_transport_modules.icnn_modules import *

In [2]:
convex_f = Simple_Feedforward_3Layer_ICNN_LastInp_Quadratic(3, 1024, "leaky_relu")
convex_g = Simple_Feedforward_3Layer_ICNN_LastInp_Quadratic(3, 1024, "leaky_relu")

In [3]:
convex_g.load_state_dict(torch.load("/Users/adrish/research/582/icnn/OT-ICNN/High_dim_experiments/Result_Sphere_Torus/convex_g_epoch_16.pt",  map_location=torch.device('cpu')))
convex_f.load_state_dict(torch.load("/Users/adrish/research/582/icnn/OT-ICNN/High_dim_experiments/Result_Sphere_Torus/convex_f_epoch_16.pt",  map_location=torch.device('cpu')))

<All keys matched successfully>

In [4]:
convex_f

Simple_Feedforward_3Layer_ICNN_LastInp_Quadratic(
  (fc1_normal): Linear(in_features=3, out_features=1024, bias=True)
  (activ_1): LeakyReLU(negative_slope=0.2)
  (fc2_normal): Linear(in_features=3, out_features=1024, bias=True)
  (fc2_convex): ConvexLinear(in_features=1024, out_features=1024, bias=False)
  (activ_2): LeakyReLU(negative_slope=0.2)
  (fc3_normal): Linear(in_features=3, out_features=1024, bias=True)
  (fc3_convex): ConvexLinear(in_features=1024, out_features=1024, bias=False)
  (activ_3): LeakyReLU(negative_slope=0.2)
  (last_convex): ConvexLinear(in_features=1024, out_features=1, bias=False)
  (last_linear): Linear(in_features=3, out_features=1, bias=True)
)

In [5]:
# def get_sphere_samples(num_samples=1000):
#     point_cloud = []
#     for i in range(num_samples):
#         theta, phi = torch.empty(2).uniform_(0, 2 * np.pi)
#         x = np.sin(theta) * np.cos(phi)
#         y = torch.cos(theta) * torch.cos(phi)
#         z = torch.sin(phi)
#         point_cloud.append(torch.stack([x, y, z]))
#     point_cloud = torch.vstack(point_cloud)
#     return point_cloud

# def get_torus_samples(num_samples=1000):
#   point_cloud = []
#   for i in range(num_samples):
#     theta, phi = torch.empty(2).uniform_(0, 2 * np.pi)
#     x = (1 + 0.5 * torch.cos(theta)) * torch.cos(phi)
#     y = (1 + 0.5 * torch.cos(theta)) * torch.sin(phi)
#     z = 0.5 * torch.sin(theta)
#     point_cloud.append(torch.stack([x, y, z]))
#   point_cloud = torch.vstack(point_cloud)
#   return point_cloud

# torus_samples = torch.autograd.Variable(get_torus_samples(), requires_grad=True)
# samples = torch.autograd.Variable(get_sphere_samples(), requires_grad=True)

In [6]:
def get_volume_samples(num_samples, condition):
    samples = []
    while True:
        sample = torch.empty(3).uniform_(-3, 3)
        if condition(sample):
            samples.append(sample)
        if len(samples) == num_samples:
            return torch.vstack(samples)
def sphere_condition(sample, radius=1):
    return (torch.linalg.norm(sample) - radius) <= 0

def torus_condition(sample, rh=1, rt=0.5):
    q = [torch.linalg.norm(sample[[0, 2]]) - rh, sample[1]]
    q = torch.stack(q)
    dist = torch.linalg.norm(q) - rt
    return dist <= 0
samples = get_volume_samples(1000, sphere_condition)
samples = torch.autograd.Variable(samples, requires_grad=True)
torus_samples = get_volume_samples(1000, torus_condition)
torus_samples = torch.autograd.Variable(torus_samples, requires_grad=True)

In [7]:
def get_scene(samples):
    return trimesh.Scene(trimesh.Trimesh(vertices=samples.detach().numpy()))

In [8]:
import time
import open3d as o3d

def animate(vis, source, target, steps = 100):
  geometry = o3d.geometry.PointCloud()
  step = 1 / steps
  while True:
      for interp in np.arange(0, 1 + step, step):
        current = (1 - interp) * source + interp * target
        geometry.points = o3d.utility.Vector3dVector(current.detach().numpy())
        geometry.paint_uniform_color(np.array([0., 0., 0.]))
        if interp == 0:
          vis.add_geometry(geometry)
        else:
          vis.update_geometry(geometry)
          vis.get_view_control().set_zoom(0.3)

          vis.poll_events()
          vis.update_renderer()
        time.sleep(0.1)

In [9]:
def show(samples):
    trimesh.PointCloud(samples.detach().numpy()).show(viewer="gl")

In [10]:
def transport(sample, func):
    potential = func(sample).sum()
    transported_samples = torch.autograd.grad(potential, sample, create_graph=True)[0]
    return transported_samples

In [ ]:
transported_samples = transport(samples, convex_f)
vis = o3d.visualization.Visualizer()
vis.create_window()
animate(vis, samples, transported_samples)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
vis.destroy_window()

In [19]:
show(transported_samples)

In [12]:
update_fn = animate(torus_samples, transported_samples, steps=100)
thr = threading.Thread(target=my_thread, args=(update_fn,))
thr.start()